# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.
Необходимо построить модель со значением метрики качества *F1* не меньше 0.75.

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Описание данных**  
Данные находятся в файле `toxic_comments.csv`. Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

**Содержание**<a id='toc0_'></a>    
- 1. [Подготовка](#toc1_)    
- 2. [Обучение](#toc2_)    
  - 2.1. [Логистическая регрессия](#toc2_1_)    
  - 2.2. [Дерево решений](#toc2_2_)    
  - 2.3. [Случайный лес](#toc2_3_)    
  - 2.4. [Градиентный бустинг](#toc2_4_)    
  - 2.5. [Тестирование модели](#toc2_5_)    
- 3. [Выводы](#toc3_)    
- 4. [Чек-лист проверки](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Подготовка](#toc0_)

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

Откроем и изучим файл

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')[['text', 'toxic']]

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Для обучения модели из датасета выберем 50_000 строк

In [5]:
df = df.sample(50_000).reset_index(drop=True)

Отделим целевой признак

In [6]:
features = df.loc[:, 'text']
target = df.loc[:, 'toxic']

Теперь проведём лемматизацию и очистку текста от лишних символов

In [7]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(X):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(X)])

def prepare_corpus(X):
    c = X.apply(lemmatize_sentence)\
         .apply(lambda k: " ".join(re.sub('\n|[^A-Za-z]', ' ', k).split()))
    return c

In [9]:
%%time
corpus = prepare_corpus(features)

CPU times: user 8min 1s, sys: 31.8 s, total: 8min 32s
Wall time: 8min 38s


Разделим данные на обучающую и тестовую выборки

In [10]:
features_train, features_test, target_train, target_test = train_test_split(corpus, target,
                                                                            test_size=0.2, random_state=325)

Изучим соотношение количества элементов в разных классах

In [11]:
target_train.value_counts()

0    35986
1     4014
Name: toxic, dtype: int64

Позитивных комментариев почти в 9 раз больше, чем негативных.

Для борьбы с дисбалансом классов воспользуемся техникой SMOTE.

Далее проведём векторизацию текста

In [12]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Вывод по разделу 1:**
- Данные загружены, получены обучающая и тестовая выборки
- Для борьбы с дисбалансом классов применена техника SMOTE
- Корпус текстов очищен от лишних символов, проведена лемматизация

## 2. <a id='toc2_'></a>[Обучение](#toc0_)

### 2.1. <a id='toc2_1_'></a>[Логистическая регрессия](#toc0_)

Обучим несколько моделей, начнём с логистической регрессии

In [ ]:
params = {
    'model__penalty': ['l2', 'l1'],
    'model__C': np.logspace(-1, 2, 4),
    'model__max_iter': [50, 100]
}

linear_pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words=stopwords)),
                                  ('smote', SMOTE(random_state=521)),
                                  ('model', LogisticRegression(solver='liblinear', random_state=342))])

linear_grid = GridSearchCV(linear_pipeline, params,
                          scoring='f1', cv=3, verbose=3, n_jobs=-1)

In [ ]:
%%time
linear_grid.fit(features_train, target_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END model__C=0.1, model__max_iter=50, model__penalty=l2;, score=0.642 total time=  10.8s
[CV 2/3] END model__C=0.1, model__max_iter=50, model__penalty=l2;, score=0.710 total time=  10.3s
[CV 3/3] END model__C=0.1, model__max_iter=50, model__penalty=l2;, score=0.649 total time=  10.8s
[CV 1/3] END model__C=0.1, model__max_iter=50, model__penalty=l1;, score=0.701 total time=   7.6s
[CV 2/3] END model__C=0.1, model__max_iter=50, model__penalty=l1;, score=0.712 total time=   7.4s
[CV 3/3] END model__C=0.1, model__max_iter=50, model__penalty=l1;, score=0.682 total time=   7.6s
[CV 1/3] END model__C=0.1, model__max_iter=100, model__penalty=l2;, score=0.642 total time=  10.0s
[CV 2/3] END model__C=0.1, model__max_iter=100, model__penalty=l2;, score=0.710 total time=   9.7s
[CV 3/3] END model__C=0.1, model__max_iter=100, model__penalty=l2;, score=0.649 total time=   9.9s
[CV 1/3] END model__C=0.1, model__max_iter=100, model_

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('smote', SMOTE(random_state=521)),
                                       ('model',
                                        LogisticRegression(random_state=342,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'model__C': array([  0.1,   1. ,  10. , 100. ]),
                         'model__max_iter': [50, 100],
                         'model__penalty': ['l2', 'l1']},
             scoring='f1', verbose=3)

In [ ]:
print('Значение F1-меры, время обучения и время оценки и при различных гиперпараметрах:')
print(pd.DataFrame(linear_grid.cv_results_)[['mean_test_score', 'mean_fit_time', 'mean_score_time']])
print('\nЛучшие гиперпараметры:\n', linear_grid.best_params_)
print('Лучшее значение F1-меры:', linear_grid.best_score_)

Значение F1-меры, время обучения и время оценки и при различных гиперпараметрах:
    mean_test_score  mean_fit_time  mean_score_time
0          0.666889       9.770961         0.878689
1          0.698115       6.697155         0.838779
2          0.666889       8.953916         0.888941
3          0.698115       6.252861         0.789112
4          0.700691      12.942302         0.808751
5          0.720901       6.086436         0.759705
6          0.700691      12.877798         0.822411
7          0.720901       5.943719         0.715254
8          0.697605      18.098349         0.760952
9          0.692067       6.330498         0.727461
10         0.697605      19.314378         0.791153
11         0.692067       6.504877         0.765999
12         0.671002      27.381119         0.769212
13         0.672496       6.089253         0.657104
14         0.671002      24.873199         0.724552
15         0.672496       6.141395         0.691661

Лучшие гиперпараметры:
 {'model__C

Значение F1-меры логистической регрессии: 0.72. Обучение занимает около 6 секунд.

### 2.2. <a id='toc2_2_'></a>[Дерево решений](#toc0_)

Теперь обучим модель решающего дерева

In [ ]:
params = {
    'model__max_depth' : list(range(4, 15, 3)),
    'model__min_samples_split' : list(range(8, 20, 4)),
    'model__min_samples_leaf' : list(range(8, 20, 4))
}

tree_pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words=stopwords)),
                                  ('smote', SMOTE(random_state=521)),
                                  ('model', DecisionTreeClassifier(random_state=342))])

tree_grid = GridSearchCV(tree_pipeline, params,
                          scoring='f1', cv=3, verbose=3, n_jobs=-1)

In [ ]:
%%time
tree_grid.fit(features_train, target_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV 1/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=8;, score=0.466 total time=   6.5s
[CV 2/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=8;, score=0.489 total time=   6.7s
[CV 3/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=8;, score=0.453 total time=   6.8s
[CV 1/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=12;, score=0.466 total time=   6.4s
[CV 2/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=12;, score=0.489 total time=   6.5s
[CV 3/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=12;, score=0.453 total time=   7.2s
[CV 1/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=16;, score=0.466 total time=   6.6s
[CV 2/3] END model__max_depth=4, model__min_samples_leaf=8, model__min_samples_split=16;, scor

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('smote', SMOTE(random_state=521)),
                                       ('model',
                                        DecisionTreeClassifier(random_state=342))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [4, 7, 10, 13],
                         'model__min_samples_leaf': [8, 12, 16],
                         'model__min_samples_split': [8, 12, 16]},
             scoring='f1', verbose=3)

In [ ]:
print('Значение F1-меры, время обучения и время оценки и при различных гиперпараметрах:')
print(pd.DataFrame(tree_grid.cv_results_)[['mean_test_score', 'mean_fit_time', 'mean_score_time']])
print('\nЛучшие гиперпараметры:\n', tree_grid.best_params_)
print('Лучшее значение F1-меры:', tree_grid.best_score_)

Значение F1-меры, время обучения и время оценки и при различных гиперпараметрах:
    mean_test_score  mean_fit_time  mean_score_time
0          0.469216       5.978589         0.645645
1          0.469216       6.039798         0.658593
2          0.469216       6.097532         0.679589
3          0.469971       5.935699         0.643538
4          0.469971       6.133486         0.769463
5          0.469971       6.786157         0.768871
6          0.469821       6.556533         0.718149
7          0.469821       6.576838         0.741103
8          0.469821       6.305274         0.684116
9          0.529332       7.164447         0.690153
10         0.529332       7.246744         0.753645
11         0.529332       6.682404         0.658416
12         0.529422       6.832504         0.695246
13         0.529422       7.075261         0.717812
14         0.529422       7.399796         0.848882
15         0.530134       7.140060         0.731055
16         0.530134       6.835978 

Решающее дерево показыает себя хуже линейной регрессии: F1 = 0.57 при времени обучения около 8 секунд.

### 2.3. <a id='toc2_3_'></a>[Случайный лес](#toc0_)

Теперь обучим случайный лес. Вместо обычного поиска по сетке используем HalvingGridSearchCV, выбрав в качестве ресурса колиество деревьев.

In [ ]:
params = {
    'model__max_depth' : [8, 10, 12],
    'model__min_samples_split' : [8, 10],
    'model__min_samples_leaf' : [12, 15]
}
rf_pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words=stopwords)),
                                  ('smote', SMOTE(random_state=521)),
                                  ('model', RandomForestClassifier(random_state=342))])

rf_grid  = HalvingGridSearchCV(rf_pipeline, params, scoring='f1',
                                  cv=3, verbose=3, n_jobs=-1, max_resources=100, factor=3,
                                  resource='model__n_estimators')

In [ ]:
%%time
rf_grid.fit(features_train, target_train)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 11
max_resources_: 100
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 12
n_resources: 11
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END model__max_depth=8, model__min_samples_leaf=12, model__min_samples_split=8, model__n_estimators=11;, score=(train=0.314, test=0.296) total time=   6.4s
[CV 2/3] END model__max_depth=8, model__min_samples_leaf=12, model__min_samples_split=8, model__n_estimators=11;, score=(train=0.346, test=0.341) total time=   6.7s
[CV 3/3] END model__max_depth=8, model__min_samples_leaf=12, model__min_samples_split=8, model__n_estimators=11;, score=(train=0.304, test=0.298) total time=   6.6s
[CV 1/3] END model__max_depth=8, model__min_samples_leaf=12, model__min_samples_split=10, model__n_estimators=11;, score=(train=0.314, test=0.296) total time=   6.4s
[CV 2/3] END model__max_depth=8, model__min_samples_leaf=12, model__min_samples_s

HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(stop_words=['i',
                                                                           'me',
                                                                           'my',
                                                                           'myself',
                                                                           'we',
                                                                           'our',
                                                                           'ours',
                                                                           'ourselves',
                                                                           'you',
                                                                           "you're",
                                                                           "you've",
                                                                           "you'll",
                                                                           "you'd",
                                                                           'your',
                                                                           'yours',
                                                                           'yourself',
                                                                           'yourselves',
                                                                           'he',
                                                                           'him',
                                                                           'his',
                                                                           'himself',
                                                                           'she',
                                                                           "she's",
                                                                           'her',
                                                                           'hers',
                                                                           'herself',
                                                                           'it',
                                                                           "it's",
                                                                           'its',
                                                                           'itself', ...])),
                                              ('smote',
                                               SMOTE(random_state=521)),
                                              ('model',
                                               RandomForestClassifier(random_state=342))]),
                    max_resources=100, n_jobs=-1,
                    param_grid={'model__max_depth': [8, 10, 12],
                                'model__min_samples_leaf': [12, 15],
                                'model__min_samples_split': [8, 10]},
                    resource='model__n_estimators', scoring='f1', verbose=3)

In [ ]:
params_names = ['n_resources', 'param_model__max_depth','param_model__min_samples_split', 'param_model__min_samples_leaf']

res = pd.DataFrame(rf_grid.cv_results_)[['mean_fit_time', 'mean_score_time', 'mean_test_score', *params_names]]
best_res = res[res.mean_test_score == res.mean_test_score.max()].iloc[0]

print('Лучшая модель')
print('F1-мера:', float(best_res['mean_test_score']))
display('Гиперпараметры:', best_res[params_names])
display('Время обучения и время предсказания:', best_res[['mean_fit_time', 'mean_score_time']])

Лучшая модель
F1-мера: 0.4578692921820453


'Гиперпараметры:'

n_resources                       99
param_model__max_depth            12
param_model__min_samples_split     8
param_model__min_samples_leaf     12
Name: 16, dtype: object

'Время обучения и время предсказания:'

mean_fit_time      7.320529
mean_score_time    0.952754
Name: 16, dtype: object

Лучший результат здесь даёт лес, в котором 99 деревьев. При этом значение целевой метрики уменьшается до 0.45. Время обучения составляет 7.3 сек, время предсказания - 0.95 сек.

### 2.4. <a id='toc2_4_'></a>[Градиентный бустинг](#toc0_)

Следующая модель - градиентный бустинг (библиотека LigthGBM).

In [23]:
params = {
    "model__max_depth": [12, 14],
    "model__boosting_type" : ['gbdt'],
    "model__min_child_samples" : [20],
    "model__learning_rate" : [0.1, 0.3],
    "model__num_leaves": [70]
}
lgbm_pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words=stopwords)),
                                  ('smote', SMOTE(random_state=521)),
                                  ('model', LGBMClassifier(random_state=342))])

lgbm_grid  = HalvingGridSearchCV(lgbm_pipeline, params, scoring='f1',
                                  cv=2, verbose=3, n_jobs=-1, max_resources=100, factor=3,
                                  resource='model__n_estimators')

In [24]:
%%time
lgbm_grid.fit(features_train, target_train)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 33
max_resources_: 100
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4
n_resources: 33
Fitting 2 folds for each of 4 candidates, totalling 8 fits
----------
iter: 1
n_candidates: 2
n_resources: 99
Fitting 2 folds for each of 2 candidates, totalling 4 fits
CPU times: user 1min 38s, sys: 1.18 s, total: 1min 39s
Wall time: 4min 37s


HalvingGridSearchCV(cv=2,
                    estimator=Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(stop_words=['i',
                                                                           'me',
                                                                           'my',
                                                                           'myself',
                                                                           'we',
                                                                           'our',
                                                                           'ours',
                                                                           'ourselves',
                                                                           'you',
                                                                           "you're",
                                                                           "you've",
                                                                           "you'll",
                                                                           "you'd",
                                                                           'your',
                                                                           'yours',
                                                                           'yourself',
                                                                           'yourselves',
                                                                           'he',
                                                                           'him',
                                                                           'his',
                                                                           'himself',
                                                                           'she',
                                                                           "she's",
                                                                           'her',
                                                                           'hers',
                                                                           'herself',
                                                                           'it',
                                                                           "it's",
                                                                           'its',
                                                                           'itself', ...])),
                                              ('smote',
                                               SMOTE(random_state=521)),
                                              ('model',
                                               LGBMClassifier(random_state=342))]),
                    max_resources=100, n_jobs=-1,
                    param_grid={'model__boosting_type': ['gbdt'],
                                'model__learning_rate': [0.1, 0.3],
                                'model__max_depth': [12, 14],
                                'model__min_child_samples': [20],
                                'model__num_leaves': [70]},
                    resource='model__n_estimators', scoring='f1', verbose=3)

In [26]:
params_names = ['n_resources', 'param_model__max_depth', 'param_model__min_child_samples', 'param_model__num_leaves',
                'param_model__learning_rate']

res = pd.DataFrame(lgbm_grid.cv_results_)[['mean_fit_time', 'mean_score_time', 'mean_test_score', *params_names]]
best_res = res[res.mean_test_score == res.mean_test_score.max()]

print('Лучшая модель')
print('F1:', float(best_res['mean_test_score']))
display('Гиперпараметры:', best_res[params_names])
display('Время обучения и время предсказания:', best_res[['mean_fit_time', 'mean_score_time']])

Лучшая модель
F1: 0.7165395426881531


'Гиперпараметры:'

,n_resources,param_model__max_depth,param_model__min_child_samples,param_model__num_leaves,param_model__learning_rate
4,99,12,20,70,0.3


'Время обучения и время предсказания:'

,mean_fit_time,mean_score_time
4,35.977067,1.85747


Градиентный бустинг выдаёт высокое качество (0.71), но при этом обучается дольше (36 сек.).

Сравним обученные модели по метрике качества, времени обучения и предсказания.

|Модель                        |F1-мера|Время обучения (сек.)|Время предсказания (сек.)|
|------------------------------|-------|---------------------|-------------------------|
|Линейная регрессия            |0.72   |6                    |0.71                     |
|Решающее дерево               |0.57   |8                    |0.7                      |
|Случайный лес                 |0.45   |7.3                  |0.95                     |
|Градиентный бустинг (LightGBM)|0.71   |36                   |1.85                     |

### 2.5. <a id='toc2_5_'></a>[Тестирование модели](#toc0_)

Лучше всего себя показала модель логистической регрессии. Проверим теперь её качество, обучив на всем трейне и измерив значение F1-меры на тестовой выборке.

In [ ]:
model = make_pipeline(
    TfidfVectorizer(stop_words=stopwords),
    LogisticRegression(solver='liblinear', C=3.5, max_iter=50, penalty='l1', random_state=342))
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('F1-мера на тестовой выборке:', f1_score(target_test, predictions))

F1-мера на тестовой выборке: 0.7693150684931507


**Вывод по разделу 2:**
- Обучены несколько моделей: логистическая регрессия, решающее дерево, случайный лес
- Выбрана лучшая модель: логистическая регрессия со следующими гиперпараметрами: *solver='liblinear', C=3.5, max_iter=50, penalty='l1'.*
- Качество модели проверено на тестовой выборке (значение F1-меры: 0.769)

## 3. <a id='toc3_'></a>[Выводы](#toc0_)

- Данные загружены, для борьбы с дисбалансом классов применена техника SMOTE.
- Корпус текстов очищен от лишних символов, проведена лемматизация, в качестве признаков модели взяты значения TF-IDF.
- Обучены несколько моделей: логистическая регрессия, решающее дерево, случайный лес, градиентный бустинг.
- Выбрана лучшая модель: логистическая регрессия.
- Качество модели проверено на тестовой выборке: значение F1-меры: 0.769.

## 4. <a id='toc4_'></a>[Чек-лист проверки](#toc0_)

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны